In [1]:
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

In [46]:
def rbf(x, c, s):

    #INVERSE
    #return 1.0 / np.sqrt(np.linalg.norm(x - c)**2 + s**2)

    #MULTIQUADRICS
     # return np.sqrt(np.linalg.norm(x - c)**2 + s**2)

    #GAUSSIAN
    return np.exp(-np.linalg.norm(x - c)**2 / (2 * s**2))


In [50]:
class RBFNetwork:
    def __init__(self, num_centers, num_classes, learning_rate=0.01, epochs=10):
        self.num_centers = num_centers
        self.num_classes = num_classes
        self.lr = learning_rate
        self.epochs = epochs
        self.weights = None
        self.centers = None
        self.sigma = None
        self.mse_history = []
        self.accuracy_history = []

    def train(self, X_train, y_train, centers, sigma):
        #Εκπαίδευση του δικτύου RBF
        self.centers = centers
        self.sigma = sigma
        self.weights = np.random.randn(self.num_centers, self.num_classes)

        for epoch in range(self.epochs):
            predictions = np.zeros((X_train.shape[0], self.num_classes))
            for i in range(X_train.shape[0]):
                activations = np.array([rbf(X_train[i], c, sigma) for c in centers])
                predictions[i] = activations.dot(self.weights)

                #Ενημέρωση βαρών (κανόνας δέλτα)
                error = y_train[i] - predictions[i]  #Σφάλμα:error = y_true - y_pred
                self.weights += self.lr * np.outer(activations, error)  #w = w + lr * activation * error

            #Υπολογισμός μέσης τετραγωνικής απόκλισης
            mse = np.mean((y_train - predictions)**2)
            self.mse_history.append(mse)

            # Υπολογισμός ακρίβειας εκπαίδευσης
            y_pred = np.argmax(predictions, axis=1)
            y_true = np.argmax(y_train, axis=1)
            accuracy = accuracy_score(y_true, y_pred)
            self.accuracy_history.append(accuracy)

            print(f"Epoch {epoch + 1}/{self.epochs} - MSE: {mse:.4f}, Accuracy: {accuracy:.4f}")

    def predict(self, X_test):
        #Πρόβλεψη για test δεδομένα
        predictions = np.zeros((X_test.shape[0], self.num_classes))
        for i in range(X_test.shape[0]):
            activations = np.array([rbf(X_test[i], c, self.sigma) for c in self.centers])
            predictions[i] = activations.dot(self.weights)
        return np.argmax(predictions, axis=1)

In [40]:
def compute_sigma(centres):

    num_centres = centres.shape[0]
    dists = np.linalg.norm(centres[:, None] - centres[None, :], axis=2)
    mean_dist = np.mean(dists[np.triu_indices(num_centres, k=1)])

    sigma = mean_dist / np.sqrt(2 * num_centres)
    return sigma

In [41]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Κανονικοποίηση
    transforms.Lambda(lambda x: x.view(-1))  # Flatten εικόνων
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Μετατροπή σε numpy arrays
X_train = np.array([train_dataset[i][0].numpy() for i in range(len(train_dataset))])
y_train = np.array([train_dataset[i][1] for i in range(len(train_dataset))])
X_test = np.array([test_dataset[i][0].numpy() for i in range(len(test_dataset))])
y_test = np.array([test_dataset[i][1] for i in range(len(test_dataset))])

Files already downloaded and verified
Files already downloaded and verified


In [42]:
pca = PCA(n_components=0.90, svd_solver='full')
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [7]:
def k_means(X, k, max_iters=100, tol=1e-8, random_state=None):

    if random_state is not None:
        np.random.seed(random_state)

    #Αρχικοποίηση των κέντρων
    n_samples, n_features = X.shape
    centroids = X[np.random.choice(n_samples, k, replace=False)]

    for iteration in range(max_iters):
        distances = np.linalg.norm(X[:, None] - centroids[None, :], axis=2)

       # κατηγοριοποίηση των xi
        labels = np.argmin(distances, axis=1)

        #Υπολογισμός νέων κέντρων
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])

        #Έλεγχος για σύγκλιση
        if np.max(np.linalg.norm(new_centroids - centroids, axis=1)) < tol:
            print(f"Σύγκλιση μετά από {iteration + 1} επαναλήψεις.")
            break

        centroids = new_centroids

    return centroids

In [8]:
def select_centers(X, num_centers, method , random_state=None):

    if method == 'kmeans':

        centers = k_means(X_train_pca, 50 , max_iters=100 , tol=1e-4, random_state=42)

    elif method == 'random':
        np.random.seed(random_state)
        centers = X[np.random.choice(X.shape[0], num_centers, replace=False)]
    else:
        raise ValueError("Wrong method. Use 'kmeans' or 'random'.")
    return centers

In [ ]:
def select_centers(X, num_centers, method , random_state=None):
# Έτοιμος κώδικας για k-means

    if method == 'kmeans':
        kmeans=KMeans(n_clusters=num_centers, random_state=random_state)
        kmeans.fit(X)
        centers = kmeans.cluster_centers_

    elif method == 'random':
        np.random.seed(random_state)
        centers = X[np.random.choice(X.shape[0], num_centers, replace=False)]
    else:
        raise ValueError("Wrong method. Use 'kmeans' or 'random'.")
    return centers

In [59]:
num_centers = 50

centers = select_centers(X_train, num_centers, method='random', random_state=42)
#centers = [np.array(c) for c in centers]
#centers = np.array(centers, dtype=object)

In [60]:
sigma = compute_sigma(centers)
sigma = 2*sigma
print(f"Computed Sigma: {sigma:.4f}")

# One-Hot Encoding
num_classes = 10
#y_train_labels = np.argmax(y_train, axis=1).astype(int)
y_train_one_hot = np.eye(num_classes)[y_train]
#y_test_labels = np.argmax(y_test, axis=1).astype(int)
y_test_one_hot = np.eye(num_classes)[y_test]

# Εκπαίδευση
rbf_net = RBFNetwork(num_centers=num_centers, num_classes=num_classes, learning_rate=0.01, epochs=20)
rbf_net.train(X_train, y_train_one_hot, centers, sigma)

#Αξιολόγηση
y_pred = rbf_net.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)  # Ακρίβεια
print(f"Test Accuracy: {test_accuracy:.4f}")

Computed Sigma: 7.4739
Epoch 1/20 - MSE: 0.1029, Accuracy: 0.0768
Epoch 2/20 - MSE: 0.1003, Accuracy: 0.1026
Epoch 3/20 - MSE: 0.0988, Accuracy: 0.1321
Epoch 4/20 - MSE: 0.0979, Accuracy: 0.1668
Epoch 5/20 - MSE: 0.0973, Accuracy: 0.1836
Epoch 6/20 - MSE: 0.0968, Accuracy: 0.1888
Epoch 7/20 - MSE: 0.0965, Accuracy: 0.1916
Epoch 8/20 - MSE: 0.0962, Accuracy: 0.1940
Epoch 9/20 - MSE: 0.0960, Accuracy: 0.1964
Epoch 10/20 - MSE: 0.0958, Accuracy: 0.1979
Epoch 11/20 - MSE: 0.0956, Accuracy: 0.1997
Epoch 12/20 - MSE: 0.0955, Accuracy: 0.2010
Epoch 13/20 - MSE: 0.0954, Accuracy: 0.2021
Epoch 14/20 - MSE: 0.0953, Accuracy: 0.2032
Epoch 15/20 - MSE: 0.0952, Accuracy: 0.2045
Epoch 16/20 - MSE: 0.0951, Accuracy: 0.2055
Epoch 17/20 - MSE: 0.0950, Accuracy: 0.2064
Epoch 18/20 - MSE: 0.0950, Accuracy: 0.2071
Epoch 19/20 - MSE: 0.0949, Accuracy: 0.2077
Epoch 20/20 - MSE: 0.0948, Accuracy: 0.2082
Test Accuracy: 0.2145


In [ ]:

#ΕΤΟΙΜΟΣ ΚΩΔΙΚΑΣ ΓΙΑ ΑΝΑΠΑΡΑΣΤΑΣΗ
import matplotlib.pyplot as plt
X_test = (X_test + 1) * 127.5  # Επαναφορά από [-1, 1] σε [0, 255]
X_test = X_test.astype('uint8')  # Μετατροπή σε ακέραιες τιμές

def plot_classification_results(X_test, y_test, y_pred, num_examples=5):
    """
    Οπτικοποιεί παραδείγματα από σωστές και λανθασμένες ταξινομήσεις.
    """
    correct_indices = np.where(y_test == y_pred)[0]
    incorrect_indices = np.where(y_test != y_pred)[0]

    # Σωστές ταξινομήσεις
    print("Correct Classifications:")
    plt.figure(figsize=(10, 5))
    for i in range(min(num_examples, len(correct_indices))):
        idx = correct_indices[i]
        plt.subplot(2, num_examples, i + 1)
        # Use the original X_test for visualization, not the PCA transformed data
        plt.imshow(X_test[idx].reshape(32, 32, 3))  # Αναμόρφωση εικόνας 32x32x3
        plt.title(f"Pred: {y_pred[idx]}, True: {y_test[idx]}")
        plt.axis('off')

    # Λανθασμένες ταξινομήσεις
    print("Incorrect Classifications:")
    for i in range(min(num_examples, len(incorrect_indices))):
        idx = incorrect_indices[i]
        plt.subplot(2, num_examples, num_examples + i + 1)
        # Use the original X_test for visualization, not the PCA transformed data
        plt.imshow(X_test[idx].reshape(32, 32, 3))
        plt.title(f"Pred: {y_pred[idx]}, True: {y_test[idx]}")
        plt.axis('off')

    plt.show()

# Κλήση της συνάρτησης, using the original X_test
plot_classification_results(X_test, y_test, y_pred)